<span type="title">Spring JDBC and Transaction</span> | <span type="update">2018-10-09</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Spring 对于 JDBC 和事务的支持。</p></span>

# JDBCTemplate

JDBCTemplate 是一个类似于 DBUtils 的 JDBC 支持工具，而并非 ORM 框架。在 Spring 中，直接使用 JDBCTemplate 来编写 DAO，而不是使用 DBUtils 更加的方便，这两者使用方法类似，但是也有差别。

首先，在 Spring 中使用数据库一般需要通过 XML 手动声明 bean 或者注解自动扫描的方式进行。不论哪一种情况，都需要创建 DataSource 数据源，使用 JDBCTemplate 数据库操纵工具来管理数据源。一个通过 XML 手动声明 bean 的例子如下：

```xml
<bean id="dataSource" class="com.mchange.v2.c3p0.ComboPooledDataSource">
    <property name="user" value="corkine" />
    <property name="password" value="password" />
    <property name="jdbcUrl" value="jdbc:mysql:///log" />
    <property name="driverClass" value="com.mysql.jdbc.Driver" />
    <property name="initialPoolSize" value="5" />
    <property name="maxPoolSize" value="10" />
</bean>

<bean id="temp" class="org.springframework.jdbc.core.JdbcTemplate">
    <property name="dataSource" ref="dataSource"/>
</bean>
```

之后初始化 context，加载 bean 即可：

```java
ApplicationContext context = new ClassPathXmlApplicationContext("spring-temp.xml");
JDBCTemplate template = (JdbcTemplate) context.getBean("temp");
```

## Update

因为 JDBCTemplate 集成了 DataSource，因此不用像 DBUtils 那样传入连接才能进行 update。使用 JDBCTemplate 进行更新（没有返回值的任何数据库操作，包括更新、删除和插入）如下：

```java
String sql = "update customers set name = ? where id = ?";
template.update(sql,"Marvin",10522);
```

批量更新采用 `batchUpdate` 方法，这里需要注意，传入参数是一个数组列表，其中列表对应批量的语句，数组对应每条语句的占位符。

```java
String sql = "update customers set name = ? where id = ?";
List<Object[]> list = new ArrayList<>();
list.add(new Object[]{"Lili",10523});
list.add(new Object[]{"Lili2",10524});
list.add(new Object[]{"Lili3",10525});
template.batchUpdate(sql,list);
```

## Query

**使用 queryForObject(sql, rowMapper) 查询一个对象**

查询并且返回对象，不需要使用 BeanUtils 等工具手动构建，而是需要指定对象的类，采用 `queryForObject` 方法进行对象查询并返回，注意，此方法只返回对象，并且只返回一个对象。

```java
String sql = "select * from customers where id = ?";
RowMapper<Customer> mapper = new BeanPropertyRowMapper<>(Customer.class);
Customer customer = template.queryForObject(sql,mapper,521);
System.out.println(customer);
```

注意查询需要传入一个 RowMapper，这个 RowMapper 的泛型用于强行转换最后返回的类型，在这里是 Customer 类型，这个 RowMapper 接受的构造器参数类型将用来作为实际装载数据库信息的 beans。这样的分配看起来很奇怪，但是如果你想要使用一种类型来装载 beans，而获取另一种引用，这种方法就很不错了。

此外，如果想要获取一组对象的话，这里的 RowMapper 不需要改变，但是方法变了！！！！使用 query 方法查询多个对象。

**使用 queryForObject(sql, Class<T\>) 查询一个值**

此外还需要区分 `queryForObject（sql，Class<T>）` 方法，此方法用来将查询的值转换为参数中的对象类型（即用来强制转换的，而不是作为 bean 构造的），此方法用于获取单个数据库值，并转换成指定类型。

```java
String sql = "select count(id) from customers";
Long res = template.queryForObject(sql,Long.class);
System.out.println(res);
```

**使用 query(sql, rowMapper) 查询多个对象**

如果需要查询并返回一个列表的对象，使用 query 方法：

```java
String sql = "select * from customers where id < ?";
RowMapper<Customer> mapper = new BeanPropertyRowMapper<>(Customer.class);
List<Customer> list = template.query(sql,mapper,1000);
System.out.println(list);
```

注意，在这种情况下，我们依然使用的是 `BeanPropertyRowMapper` 方法，在 JDBCTempalte 中的 mapper 没有返回列表、Map等不同结构的包装器，这和 DBUtils 对于列表对象返回采用不同的 ResultRowHandler 不同。在 JDBCTemplate 中，使用 query 方法而不是 queryForObject 方法来返回列表对象，以此作区分。

注意，查询多个对象不要使用 `queryForList` 方法。

**总结**

总而言之，rowMapper 没有多种包装器，这是 JDBCTemplate 和 DBUtils 最大的不同。当查询单个值/对象时，使用 queryForObject，此方法传入类型参数可返回一个值的类型，传入 rowMapper 可构造为指定类型 bean 并转型为指定泛型类型。当查询多个对象时，使用 query 方法，传入 rowMapper 即可。

# NamedParameterJdbcTemplate

具名的 JDBCTemaple 在构造 bean 时需要向构造器传入参数，并且其没有无参构造器，这是和普通的 JdbcTemplate 不同的地方(普通版本是通过 property 方式传入数据源的)：

```xml
<bean id="nTemp" class="org.springframework.jdbc.core.namedparam.NamedParameterJdbcTemplate">
    <constructor-arg ref="dataSource" />
</bean>
```

```java
ntemplate = (NamedParameterJdbcTemplate) context.getBean("nTemp");
```

## Query and Update

具名的模板工具更新起来的优点在于，可以使用 `:abc` 来替代问号的占位符，这样的好处在于，如果一个 SQL 语句有很多占位符，那么采用引用的方式更加清晰一些。

```java
String sql = "insert into customers (id,name,email,birth) values (:id,:name,:email,:birth)";
Map<String,Object> map = new HashMap<>();
map.put("id",33);
map.put("name","Corkine Ma");
map.put("email","cm@muninn.cc");
map.put("birth",new Date(new java.util.Date().getTime()));
ntemplate.update(sql,map);
```

更新时需要传入一个 Map，此 Map 的 key 为引用名称， value 为对应需要填充的值。

此外，我们可以传入一个对象，从这个对象身上拔取属性自动填入占位符：

```java
String sql = "insert into customers (id,name,email,birth) values (:id,:name,:email,:birth)";
Customer obj = new Customer(40,"Lili","li@muninn.cn",new Date(new java.util.Date().getTime()));
SqlParameterSource psource = new BeanPropertySqlParameterSource(obj);
ntemplate.update(sql,psource);
```

在这种情况下，我们需要传入 `SqlParameterSource` 这个对象进行更新，而这个对象则通过 `BeanPropertySqlParameterSource` 这个实现类来接受一个 bean，然后从这个 bean 上自动拔取属性填入占位符要求的属性字段。

# JdbcDaoSupport and DAO

JdbcDaoSupport 希望我们在自己编写 DAO 的时候继承 JdbcDaoSupport 类，这个类提供了 getDataSource 的方法，封装很简单。但是继承这个类依旧需要在构造器传入 DataSource，因此颇为鸡肋，一般不使用。

一个基本的 DAO，在 Spring 中如下形式：

```java
@Repository
public class CustomerDao {
    private JdbcTemplate template;

    @Autowired
    public CustomerDao(JdbcTemplate template) {
        this.template = template;
    }

    public Customer get(Integer id) {
        String sql = "select * from customers where id = ?";
        RowMapper<Customer> mapper = new BeanPropertyRowMapper<>(Customer.class);
        Customer customer = template.queryForObject(sql,mapper,id);
        return customer;
    }
}
```

因为 Spring 官方不建议直接装配私有方法（装配私有方法造成 Spring 的侵入性太强），因此这里我们使用了构造器来进行装配（或者可以使用 getter/setter）。

注意，开启 DAO 的注解自动装配，需要启用注解自动扫描，对于 XML 手动配置而言，需要声明：

```xml
<context:component-scan base-package="com.mazhangjing.spring.jdbc" resource-pattern="*.class" >
</context:component-scan>
```

# Spring  Transaction

## 基于 @Transcational 的注解

